In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import mysql.connector
from mysql.connector import Error

In [6]:
url = 'https://www.speedtest.net/global-index'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

In [17]:
soup.find('table', class_ = 'data-result results')

[]

In [19]:
soup.find_all('th')

[<th class="rank" colspan="2" width="20%">#</th>,
 <th class="labelType" width="45%">Country</th>,
 <th class="speed" width="35%"><svg class="svg-icon svg-icon-larger"><use xlink:href="#icon-download"></use></svg><span class="unit list-unit">Mbps</span></th>,
 <th class="rank" colspan="2" width="20%">#</th>,
 <th class="labelType" width="45%">Country</th>,
 <th class="speed" width="35%"><svg class="svg-icon svg-icon-larger"><use xlink:href="#icon-download"></use></svg><span class="unit list-unit">Mbps</span></th>,
 <th class="rank" colspan="2" width="20%">#</th>,
 <th class="labelType" width="45%">Country</th>,
 <th class="speed" width="35%"><svg class="svg-icon svg-icon-larger"><use xlink:href="#icon-download"></use></svg><span class="unit list-unit">Mbps</span></th>,
 <th class="rank" colspan="2" width="20%">#</th>,
 <th class="labelType" width="45%">City</th>,
 <th class="speed" width="35%"><svg class="svg-icon svg-icon-larger"><use xlink:href="#icon-download"></use></svg><span clas

In [7]:

soup = BeautifulSoup(page.text, 'lxml')

In [8]:
soup.find_all('th',class_='labelType')

[<th class="labelType" width="45%">Country</th>,
 <th class="labelType" width="45%">Country</th>,
 <th class="labelType" width="45%">Country</th>,
 <th class="labelType" width="45%">City</th>,
 <th class="labelType" width="45%">Country</th>,
 <th class="labelType" width="45%">City</th>]

In [9]:
for i in soup.find_all('th',class_='labelType'):
     print(i.text.strip())

Country
Country
Country
City
Country
City


In [41]:
table=soup.find_all('table',class_='list-results')

In [42]:
data=soup.find_all('tr',class_='data-result results')

In [14]:
len(data)

897

In [16]:
data=soup.find_all('tr',class_='data-result results')


In [17]:
mobile_countries = []
fixed_countries = []
mobile_rank = []
fixed_rank = []
mobile_Mbps = []
fixed_Mbps = []

for i in data:
    country_td = i.find('td', class_='country')
    if country_td:
        country_a = country_td.find('a')
        if country_a:
            href = country_a.get('href')
            country_name = country_a.text.strip()
            rank_val = i.find('td',class_='rank actual-rank').text.strip()
            Mbps_val = i.find('td',class_='speed').text.strip()
            if '#mobile' in href:
                mobile_countries.append(country_name)
                mobile_rank.append(rank_val)
                mobile_Mbps.append(Mbps_val)
            elif '#fixed' in href:
                fixed_countries.append(country_name)
                fixed_rank.append(rank_val)
                fixed_Mbps.append(Mbps_val)

# create DataFrames
df_mobile = pd.DataFrame({'Rank': mobile_rank,'Country': mobile_countries, 'Mbps': mobile_Mbps})
df_fixed = pd.DataFrame({'Rank': fixed_rank,'Country': fixed_countries, 'Mbps': fixed_Mbps})

print(df_mobile)
print(df_fixed)

    Rank                  Country    Mbps
0      1                    Qatar  473.58
1      2     United Arab Emirates  462.61
2      3              South Korea  285.04
3      4                   Kuwait  277.67
4      5                  Denmark  232.84
..   ...                      ...     ...
372  157                    Gomel   13.62
373  158  Santa Cruz de la Sierra   12.29
374  159                   La Paz    9.67
375  160                     Aden    8.79
376  161                   Havana    4.47

[377 rows x 3 columns]
    Rank               Country    Mbps
0      1             Singapore  385.29
1      2               Iceland  368.21
2      3                France  366.20
3      4       Hong Kong (SAR)  358.30
4      5  United Arab Emirates  342.88
..   ...                   ...     ...
515  198                 Herat    3.37
516  199                 Kabul    3.37
517  200              Damascus    2.88
518  201                Havana    2.55
519  202                Aleppo    1.87

[52

In [18]:
df_mobile.to_csv('mobile_data.csv', index=False)
df_fixed.to_csv('fixed_data.csv', index=False)



In [19]:
df_mobile

,Rank,Country,Mbps
0,1,Qatar,473.58
1,2,United Arab Emirates,462.61
2,3,South Korea,285.04
3,4,Kuwait,277.67
4,5,Denmark,232.84
...,...,...,...
372,157,Gomel,13.62
373,158,Santa Cruz de la Sierra,12.29
374,159,La Paz,9.67
375,160,Aden,8.79


In [20]:
df_fixed

,Rank,Country,Mbps
0,1,Singapore,385.29
1,2,Iceland,368.21
2,3,France,366.20
3,4,Hong Kong (SAR),358.30
4,5,United Arab Emirates,342.88
...,...,...,...
515,198,Herat,3.37
516,199,Kabul,3.37
517,200,Damascus,2.88
518,201,Havana,2.55


In [37]:
try:
    db = mysql.connector.connect(
        host="localhost",  
        user="root",  
        password="Priya@850",  
        database="tutorial"  
    )
    print("Connection established successfully!")

    cursor = db.cursor()

    # Create tables
    create_mobile_table_query = """
    CREATE TABLE IF NOT EXISTS mobile_data (
        id INT AUTO_INCREMENT PRIMARY KEY,
        country VARCHAR(255),
        `rank` VARCHAR(255),
        mbps VARCHAR(255)
    )
    """
    create_fixed_table_query = """
    CREATE TABLE IF NOT EXISTS fixed_data (
        id INT AUTO_INCREMENT PRIMARY KEY,
        country VARCHAR(255),
        `rank` VARCHAR(255),
        mbps VARCHAR(255)
    )
    """
    
    cursor.execute(create_mobile_table_query)
    print("mobile_data table created successfully!")
    
    cursor.execute(create_fixed_table_query)
    print("fixed_data table created successfully!")

    # Insert data into mobile_data table
    for index, row in df_mobile.iterrows():
        cursor.execute("""
          INSERT INTO mobile_data (country, `rank`, mbps)
          VALUES (%s, %s, %s)
        """, (row['Country'], row['Rank'], row['Mbps']))
    print("Data inserted into mobile_data table successfully!")

    # Insert data into fixed_data table
    for index, row in df_fixed.iterrows():
        cursor.execute("""
          INSERT INTO fixed_data (country, `rank`, mbps)
          VALUES (%s, %s, %s)
        """, (row['Country'], row['Rank'], row['Mbps']))
    print("Data inserted into fixed_data table successfully!")

    
    db.commit()
    print("Changes committed successfully!")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    
    if cursor:
        cursor.close()
    if db:
        db.close()
    print("Connection closed.")

  
   

  

Connection established successfully!
mobile_data table created successfully!
fixed_data table created successfully!
Data inserted into mobile_data table successfully!
Data inserted into fixed_data table successfully!
Changes committed successfully!
Connection closed.
